# Section 7

In [17]:
using PyPlot

rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams")

rcParams["text.usetex"] = true
rcParams["font.family"] = "serif"
rcParams["font.size"] = 16
rcParams["text.latex.preamble"] = raw"\usepackage{amsmath}"

"\\usepackage{amsmath}"

In [18]:
import PyCall: pyimport
irbasis3 = pyimport("irbasis3")

PyObject <module 'irbasis3' from '/Users/hiroshi/git/irbasis3/src/irbasis3/__init__.py'>

## Generate IR basis by singular value expansion

In [19]:
# Create kenerl for lambda_ = 100 and beta = 10
# The default value of eps ``cutoff for singular values" is
# set to 2.2e-16.
# In this sample, we compute fewer basis functions.
lambda_ = 100
beta = 10
wmax = lambda_/beta
K = irbasis3.KernelFFlat(lambda_=100)
basis = irbasis3.FiniteTempBasis(
    K, statistics="F", beta=beta, eps=1e-10)

PyObject <irbasis3.basis.FiniteTempBasis object at 0x2a4d189d0>

## Fitting $\rho(\omega)$

In [20]:
# Three Gaussian peaks (normalized to 1)
gaussian(x, mu, sigma) = exp.(-((x.-mu)/sigma).^2)/(sqrt(π)*sigma)

rho(omega) = 0.2*gaussian(omega, 0.0, 0.15) + 
    0.4*gaussian(omega, 1.0, 0.8) + 0.4*gaussian(omega, -1.0, 0.8)

omegas = collect(range(-5, 5, length=1000))
plt.xlabel(L"$\omega$")
plt.ylabel(L"$\rho(\omega)$")
plt.plot(omegas, rho(omegas))

1-element Vector{PyCall.PyObject}:
 PyObject <matplotlib.lines.Line2D object at 0x2a4d48910>

In [21]:
rhol = basis.v.overlap(rho)
plt.semilogy(1:2:length(basis.s), abs.(rhol[1:2:end]), marker="o")
plt.xlabel(L"$l$")
plt.ylabel(L"$|\rho_l|$")

PyObject Text(0, 0.5, '$|\\rho_l|$')

In [22]:
gl = -basis.s .* rhol
plt.semilogy(1:2:length(basis.s), abs.(gl[1:2:end]), marker="o")
plt.xlabel(L"$l$")
plt.ylabel(L"$|g_l|$")

PyObject Text(0, 0.5, '$|g_l|$')

## From IR to $\tau$

In [23]:
function eval_gtau(taus)
    uval = basis.u(taus)
    return transpose(uval) * gl
end

eval_gtau (generic function with 1 method)

In [24]:
taus = collect(range(0, beta, length=1000))
uval = basis.u(taus)
gtaus = transpose(uval) * gl
plt.plot(taus, gtaus)
plt.xlabel(L"$\tau$")
plt.ylabel(L"$G(\tau)$")

PyObject Text(0, 0.5, '$G(\\tau)$')

## Reconstruct $g_l$ by numerical integration

$$
g_l = \int_0^\beta d\tau U_l(\tau) G(\tau)
$$

In [25]:
gl_reconst = basis.u.overlap(eval_gtau)

30-element Vector{Float64}:
 -0.6920636785862002
 -1.3014529587296488e-17
  0.1298138445620059
 -3.8441292775924665e-17
  0.032784004824917515
  9.586397245349238e-18
  0.00047491259960263894
 -1.2606749467088898e-18
 -0.004408753068411987
 -6.702634493002293e-17
  ⋮
 -1.406014918985815e-17
  1.119088487331689e-8
  2.5337456771641985e-17
 -1.6360664398086468e-9
 -1.7618257432785552e-18
  2.0110880980903573e-10
  1.0498866063055828e-17
 -2.1923569800568072e-11
  3.1826760612787504e-17

## Reconstruct $g_l$ by sparse sampling (fitting)

In [26]:
smpl = irbasis3.TauSampling(basis)
println("Sampling points: ", smpl.sampling_points)
println("Condition number: ", smpl.cond)
gl_reconst_sparse = smpl.fit(eval_gtau(smpl.sampling_points))

Sampling points: [0.012540375107178717, 0.025080750214357433, 0.084608404998333, 0.1796989521454872, 0.31247002491447273, 0.4859594077678564, 0.7042195121898956, 0.9723734269125226, 1.2965285468854475, 1.6833693532706349, 2.1391650644480844, 2.6679119879113684, 3.268556407887193, 3.931899929356174, 4.638747103136412, 5.361252896863588, 6.068100070643824, 6.731443592112809, 7.332088012088631, 7.860834935551916, 8.316630646729367, 8.703471453114553, 9.027626573087478, 9.295780487810104, 9.514040592232144, 9.687529975085527, 9.820301047854514, 9.915391595001667, 9.974919249785641, 9.987459624892821]
Condition number: 5.89770217368647


30-element Vector{Float64}:
 -0.692063678586201
  8.910192898161881e-17
  0.12981384456200604
  6.704282595044304e-17
  0.03278400482491764
  3.988194723636315e-17
  0.00047491259960281794
 -3.768508595036529e-17
 -0.004408753068412242
 -4.654129405269268e-17
  ⋮
  3.7433444340347394e-17
  1.1190884698599271e-8
 -7.88720584629694e-18
 -1.6360665180969684e-9
  2.7754697875007994e-17
  2.011088278452334e-10
  2.2686313734929284e-17
 -2.1923455346143264e-11
 -2.458777469980358e-17

In [27]:
dim = basis.size
plt.plot(taus, basis.u[dim](taus))
plt.plot(taus, zero(taus), ls="-", color="k", marker="", lw=0.5)
for x in smpl.sampling_points
    plt.axvline(x=x, ls="--", lw=0.5, color="k")
end
plt.xlabel(L"$\tau$")
plt.ylabel(L"$U_l(\tau)$")
plt.xlim([0, beta])
plt.tight_layout()
plt.savefig("sampling_points_tau_julia.pdf")

## Comparison

In [28]:
gl = -basis.s .* rhol
plt.semilogy(abs.(gl[1:2:end]), marker="o")
plt.semilogy(abs.(gl_reconst[1:2:end]), marker="x")
plt.semilogy(abs.(gl_reconst_sparse[1:2:end]), marker="+")
plt.xlabel(L"$l$")
plt.ylabel(L"$|g_l|$")

PyObject Text(46.139200221392, 0.5, '$|g_l|$')

## Sparse sampling in Matsubara

In [29]:
function eval_giv(v)
    uhat_val = basis.uhat(v)
    return transpose(uhat_val) * gl
end

smpl_matsu = irbasis3.MatsubaraSampling(basis)
println("Sampling frequencies: ", smpl_matsu.sampling_points)
println("Condition number: ", smpl_matsu.cond)
gl_reconst_sparse_matsu = smpl_matsu.fit(eval_giv(smpl_matsu.sampling_points))

gl = -basis.s .* rhol
plt.semilogy(abs.(gl[1:2:end]), marker="o", label="Exact")
plt.semilogy(abs.(gl_reconst_sparse[1:2:end]), marker="x",
    label="Sparse sampling (tau)", color="orange")
plt.semilogy(abs.(gl_reconst_sparse_matsu[1:2:end]), marker="v",
    label="Sparse sampling (Matsubara)", color="green")
plt.semilogy(abs.(gl_reconst_sparse[1:2:end]-gl[1:2:end]), marker="", ls="--", label="",
    color="orange")
plt.semilogy(abs.(gl_reconst_sparse_matsu[1:2:end]-gl[1:2:end]), marker="", ls="--", label="",
    color="green")
plt.legend(loc="best", frameon=false)
plt.xlabel(L"$l$")
plt.ylabel(L"$|g_l|$")
plt.tight_layout()
plt.savefig("gl_sparse_julia.pdf")

Sampling frequencies: [-229, -217, -93, -55, -39, -29, -23, -17, -15, -13, -11, -9, -7, -5, -3, -1, 1, 3, 5, 7, 9, 11, 13, 15, 17, 23, 29, 39, 55, 93, 217, 229]
Condition number: 11.266380245503292


In [30]:
get_nonzero_part(l, data) = l%2 == 0 ? imag(data) : real(data)

fig, ax = plt.subplots(figsize=(10,5))
    
nmax = 2*div(maximum(smpl_matsu.sampling_points),2)
nvec = 2*collect(-nmax:nmax) .+ 1
vvec = nvec .* (pi/beta)
markers = ["o", "s"]
for (idx_l, l) in enumerate([9, basis.size-1])
    ax.plot(vvec, get_nonzero_part(l, basis.uhat[l+1](nvec)), 
    label=L"$l=" * string(l) * L"$", ls="-", marker=markers[idx_l], ms=6, lw=0.5)
end

for n in smpl_matsu.sampling_points
    ax.axvline(n*pi/beta, ls="--", lw=0.5, color="k")
    ax.axhline(0, ls="-", lw=0.25, color="k")
end

ax.legend(frameon=false)
ax.set_xlabel(L"$\nu$")
ax.set_ylabel(L"Re/Im $G(\mathrm{i}\nu)$")
    
ax.set_xlim([0.5*pi/beta, nothing])
ax.set_ylim([-10, 10])
ax.set_xscale("log")
ax.set_yscale("symlog", linthresh=1e-6)
    
plt.tight_layout()
plt.savefig("uhat_julia.pdf")